In [1]:
import json
import configparser
import time
import tweepy
import pandas as pd

#use % to install packages instead of !. %keeps it within the ennviroment

# Lemmatization

Lemmatization explanation can be found here https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

https://spacy.io/usage/linguistic-features (Spacy)

Spacy hashtag issuehttps://stackoverflow.com/questions/43388476/how-could-spacy-tokenize-hashtag-as-a-whole

spacy with this specific dataset https://towardsdatascience.com/pre-processing-should-extract-context-specific-features-4d01f6669a7e

In [ ]:
def lemmatize(text):
    """Lemmatize the tokens.
    
    Retains more natural word forms than stemming, but assumes all
    tokens are nouns unless tokens are passed as (word, pos) tuples.
    """
    text = text.lower()
    lemmas = []
    tweet = nlp(text)

    for token in tweet:
      lemmas.append(token.lemma_)
    return lemmas
    
def remove_stopwords(tokens):
    """Remove stopwords, i.e. words that we don't want as part of our
    analysis. Defaults to the default set of nltk english stopwords.
    """   
    return [ token for token in tokens if token not in stopwords ]

def remove_links(tokens):
    """Removes http/s links from the tokens.

    This simple implementation assumes links have been kept intact as whole
    tokens. E.g. the way the Tweet Tokenizer works.
    """
    return [ t for t in tokens
            if not t.startswith("http://")
            and not t.startswith("https://")
        ]

def remove_punctuation(tokens):
    """Remove punctuation from a list of tokens.

    #Note: Not looking to remove mentions as accounts mentioned can relate to 
    whether a tweet is about an actual disaster
    """
    tokens = [t for t in tokens if t not in string.punctuation]
    return tokens

def clean_tweet(text):
  """
  clean a tweet using our helper functions
  """
  tokens = lemmatize(text)
  tokens = remove_links(tokens)
  tokens = remove_stopwords(tokens)
  tokens = remove_punctuation(tokens)
  text = " ".join(tokens)
  return text

# Setting up configuration to collect data

Attributes of a tweet -> https://developer.twitter.com/en/docs/twitter-api/v1/tweets/timelines/api-reference/get-statuses-home_timeline


In [3]:
#Secure Authentication
config = configparser.ConfigParser()
config.read("config.ini")
API_KEY = config['twitter_api']['api_key']
API_SECRET = config['twitter_api']['api_secret']
ACCESS_TOKEN = config['twitter_api']['access_token']
ACCESS_TOKEN_SECRET = config['twitter_api']['access_token_secret']

In [4]:
auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [5]:
#Get tweets that I or people I follow tweeted
public_tweets = api.home_timeline()

In [6]:
print(public_tweets[0].user.screen_name)
print(public_tweets[0].text)

C_Wibz
RT @ViciousVideos: https://t.co/MFtv7rJTpL


In [7]:
# Saving Data to CSV

columns = ["Time", "User", "Tweet"]
data = []
for tweet in public_tweets:
    data.append([tweet.created_at, tweet.user.screen_name, tweet.text])

In [8]:
print(data[0])

[datetime.datetime(2022, 8, 26, 20, 38, 37, tzinfo=datetime.timezone.utc), 'C_Wibz', 'RT @ViciousVideos: https://t.co/MFtv7rJTpL']


In [10]:
df = pd.DataFrame(data, columns=columns)

In [11]:
print(df)

                        Time             User  \
0  2022-08-26 20:38:37+00:00           C_Wibz   
1  2022-08-26 20:36:41+00:00     J_Johnson300   
2  2022-08-26 20:30:45+00:00     J_Johnson300   
3  2022-08-26 20:27:50+00:00      jadigalous_   
4  2022-08-26 20:26:06+00:00        UberFacts   
5  2022-08-26 20:21:16+00:00          ESPNCFB   
6  2022-08-26 20:15:04+00:00  _endlessloyalty   
7  2022-08-26 20:09:25+00:00        Zack_Carp   
8  2022-08-26 20:02:00+00:00             odsc   
9  2022-08-26 20:01:26+00:00      taylorbang_   
10 2022-08-26 20:00:27+00:00      multiversus   
11 2022-08-26 20:00:18+00:00      CoachK_Hall   
12 2022-08-26 19:59:18+00:00      jadigalous_   
13 2022-08-26 19:57:54+00:00      jadigalous_   
14 2022-08-26 19:55:15+00:00  SupremeDreams_1   
15 2022-08-26 19:51:00+00:00        UberFacts   
16 2022-08-26 19:50:30+00:00  SupremeDreams_1   
17 2022-08-26 19:47:31+00:00      Mr_Clutch30   
18 2022-08-26 19:45:10+00:00  randon_moore123   
19 2022-08-26 19:40:

In [48]:
#q is just the words we type to search
tweets = tweepy.Cursor(api.search_tweets, q="Aaron+Rodgers OR @AaronRodgers12 -is:retweet", 
lang="en", tweet_mode="extended").items(2)

In [71]:
tweets = tweepy.Cursor(api.search_full_archive, label="test",query="Aaron+Rodgers OR @AaronRodgers12 -is:retweet lang:en", 
fromDate='202207010000', toDate="202207260000" ).items(2)

In [72]:
list_tweets = [tweet for tweet in tweets]

HTTPException: 422 Unprocessable Entity
There were errors processing your request: Reference to invalid operator 'is:retweet'. Operator is not available in current product or product packaging. Please refer to complete available operator list at http://t.co/operators. (at position 38)

# Looking at tweet json structure and what I need to pull
- `id_str`
- `full_text`
- `entities` #to get this informatio look at the tweet example below
    - `user_mentions`
        -`screen_name`
        - `id_str`
    - `user` #info about the person who tweeted it
        -  `id_str`
        -  `screen_name`
        

In [50]:
ex_tweet = list_tweets[0]
ex_tweet

Status(_api=<tweepy.api.API object at 0x7fcdac283700>, _json={'created_at': 'Fri Aug 26 22:13:00 +0000 2022', 'id': 1563288427252137984, 'id_str': '1563288427252137984', 'full_text': 'Aaron Rodgers complimented Jordan Love, his likely heir-apparent. He’s already better than Brett Favre at that, at least.\nhttps://t.co/MhMNAy7dNT', 'truncated': False, 'display_text_range': [0, 145], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/MhMNAy7dNT', 'expanded_url': 'https://fansided.com/2022/08/26/aaron-rodgers-jordan-love-mentor-packers/?a_aid=45978', 'display_url': 'fansided.com/2022/08/26/aar…', 'indices': [122, 145]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'use

In [52]:
#About the tweet itself
print("Tweet Id:", ex_tweet.id_str)
print("Tweet Text:", ex_tweet.full_text)

#would need to loop through the user mentions to get info for each
# print("Tweet mention @ name:", ex_tweet.entities['user_mentions'][0]['screen_name'])
# print("Tweet mention id:", ex_tweet.entities['user_mentions'][0]['id_str'])

##The person who created the tweet
print("Tweet Creater ID:", ex_tweet.user.id_str)
print("Tweet Creater @:", ex_tweet.user.screen_name)

Tweet Id: 1563288427252137984
Tweet Text: Aaron Rodgers complimented Jordan Love, his likely heir-apparent. He’s already better than Brett Favre at that, at least.
https://t.co/MhMNAy7dNT
Tweet Creater ID: 466105734
Tweet Creater @: FanSidedNFL
